In [3]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Goal: Estimate acceleration profile $\hat{m}$ Using Newton-Raphson to compensate for distortion between similar point cloud

Through the first order Taylor approximation we define: 

$\mathbf{h}(\hat{m})$ as the means of voxel centers in cloud $j$ passed through the distortion operator $\mathbb{C}$ according to estimated motion profile $\hat{m}$. 

$\mathbf{H}$ is the Jacobian of $\mathbf{h}$, which is multiplied by a linear correction factor $\delta m$.

$    y_i = \mathbf{h}(\hat{m}) + \mathbf{H} \delta m $


In [226]:
#load point cloud
# no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# m_hat = np.array([-3., -2., 0., 0., 0., 0.0])

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
m_hat = np.array([3., 1, 0., 0., 0., -1])
# gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt)

#downsample
old_cloud = old_cloud[::5,:]

In [227]:
from remove_motion_basic import linear_correction_old as lc 

new_cloud = lc(old_cloud, m_hat)
# undistorted = lc(new_cloud, -m_hat)

In [228]:
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
# disp.append(Points(undistorted, c = "#CB2314"))
# disp.append(Points(old_cloud, c = "#3F5151"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Analytically calculate derivatives with SymPy

In [233]:
from sympy import *

x, y, z = symbols('x y z')

init_printing(use_unicode=True)

In [264]:
#sovles for the derivative
ans1 = diff(cos(x), x)
print(ans1)

#creates an unevaluated derivative
ans2 = Derivative(cos(x), x)
print(ans2)

-sin(x)
Derivative(cos(x), x)


In [265]:
ans3 = integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)
# print(latex(ans3)) #-> converts to LaTeX so I don't have to re-copy everything 

integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)


In [270]:
a = Matrix([[1, 2], 
            [3, 4*x]])
# A = integrate(a**2)
# print(latex(A))
integrate(a**2)

⎡               2       ⎤
⎢   7⋅x      4⋅x  + 2⋅x ⎥
⎢                       ⎥
⎢                3      ⎥
⎢   2        16⋅x       ⎥
⎢6⋅x  + 3⋅x  ───── + 6⋅x⎥
⎣              3        ⎦